In [40]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import pandas as pd
import numpy as np

def checker(wrong_options,correct_options):

    names_array=[]
    ratio_array=[]    
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
           names_array.append(wrong_option)
           ratio_array.append('100')
        else:   
            x=process.extractOne(wrong_option,correct_options,scorer=fuzz.token_set_ratio)
            names_array.append(x[0])
            ratio_array.append(x[1])
    return names_array,ratio_array

df_Original_List=pd.read_excel(r'C:\Users\IN22912959\OneDrive - Tesco\Documents\Supplier Score Card\Analysis\TFMS analysis Template 12 08 2020.xlsx',sheet_name='Commercial Data')
tqc=pd.read_excel(r'C:\Users\IN22912959\OneDrive - Tesco\Documents\Supplier Score Card\Analysis\TFMS analysis Template 12 08 2020.xlsx',sheet_name='TQC Data')

In [41]:
df_Original_List.columns

Index(['lookup', 'Origin', 'Supplier_Number', 'Supplier_Name',
       'Commercial_Director', 'Commercial_Area', 'Category_Director',
       'Category_Area', 'Buying_Controller', 'Product_Area', 'Buyer',
       'Buyer_Area', 'Junior_Buyer', 'Junior_Area', 'Active_Yes_No',
       'Geography', 'Parent_Supplier_Number_TFMS', 'Parent_Supplier_Name',
       'OBL Count', 'Count', 'Supplier Name from TQC',
       'Supplier Code from TQC', 'Supplier Number from TQC', 'Match?', 'COGS'],
      dtype='object')

In [42]:
df_Original_List=df_Original_List[df_Original_List['OBL Count']>0]
df_Original_List=df_Original_List[df_Original_List['Supplier_Number']!=99999]
# df_Original_List=df_Original_List[df_Original_List['Active_Yes_No']!='No']
len(df_Original_List)

1953

In [43]:
total_combo=df_Original_List[['Junior_Buyer','Supplier_Number']].drop_duplicates()
len(total_combo)

1953

In [44]:
total_matched_combo=df_Original_List[['Supplier_Name','Junior_Buyer']][(df_Original_List['OBL Count']>0) & (df_Original_List['Supplier Name from TQC'].notnull())].drop_duplicates()
len(total_matched_combo)

1424

In [45]:
(len(total_matched_combo)/len(total_combo))*100

72.91346646185356

In [46]:
matched=df_Original_List[(df_Original_List['OBL Count']>0) & (df_Original_List['Supplier Name from TQC'].notnull())].drop_duplicates()
matched['Parent_Junior_Combo']=matched['Parent_Supplier_Name']+matched['Junior_Buyer']
matched_combo_with_TQC_Supplier_Name=matched[['Parent_Junior_Combo','Supplier Name from TQC']].drop_duplicates()
matched_combo_with_TQC_Supplier_Name.head(2)

,Parent_Junior_Combo,Supplier Name from TQC
16,FRIESLAND FOODSSPECIALCHEES E.,FrieslandCampina
17,BAKKAVORCHILLED PIZZ A.,Bakkavor


In [47]:
matching_dict=pd.Series(matched_combo_with_TQC_Supplier_Name['Supplier Name from TQC'].values,index=matched_combo_with_TQC_Supplier_Name['Parent_Junior_Combo']).to_dict()
matching_dict.get('LAURENS HOLDINGSPOTTEDDESSERT S.')

'BAKKAVOR'

In [48]:
df_Original_List['Parent_Junior_Combo']=df_Original_List['Parent_Supplier_Name']+df_Original_List['Junior_Buyer']
non_matched=df_Original_List[(df_Original_List['OBL Count']>0) & (df_Original_List['Supplier Name from TQC'].isnull())].drop_duplicates()
non_matched['TQC_Supplier_Name_2']=non_matched['Parent_Junior_Combo']
len(non_matched)
# non_matched[non_matched['Parent_Supplier_Name']=='LAURENS HOLDINGS']

515

In [49]:
non_matched=non_matched.replace({"TQC_Supplier_Name_2": matching_dict})
non_matched[non_matched['TQC_Supplier_Name_2']=='Bakkavor']
# non_matched.to_excel('df6.xlsx')

,lookup,Origin,Supplier_Number,Supplier_Name,Commercial_Director,Commercial_Area,Category_Director,Category_Area,Buying_Controller,Product_Area,...,Parent_Supplier_Name,OBL Count,Count,Supplier Name from TQC,Supplier Code from TQC,Supplier Number from TQC,Match?,COGS,Parent_Junior_Combo,TQC_Supplier_Name_2
2662,34773VEG MEALS & ING.,Commercial,34773,BAKKAVOR SPALDING(VF,FRES H.,Fresh,PREPARED FOOD S.,Prepared Foods,CHILLE D.,Chilled,...,BAKKAVOR,1,0,NaN,NaN,NaN,no match,1.0,BAKKAVORVEG MEALS & ING.,Bakkavor


In [50]:
non_matched['flag'] = np.where((non_matched['Parent_Junior_Combo'] == non_matched['TQC_Supplier_Name_2']),'True','False')
non_matched.head()

,lookup,Origin,Supplier_Number,Supplier_Name,Commercial_Director,Commercial_Area,Category_Director,Category_Area,Buying_Controller,Product_Area,...,OBL Count,Count,Supplier Name from TQC,Supplier Code from TQC,Supplier Number from TQC,Match?,COGS,Parent_Junior_Combo,TQC_Supplier_Name_2,flag
8,54364HOMEBAK E.,Commercial,54364,SWAN MILL PAPER CO.LTD.,PACKAGE D.,Packaged,GROCER Y.,Grocery,CLASSIC GROCERY.,Classic Grocery,...,4,0,NaN,NaN,NaN,no match,7.0,SWAN MILL PAPER AND CO LTDHOMEBAK E.,SWAN MILL PAPER AND CO LTDHOMEBAK E.,True
15,64784FBAKERYORDERABL.,Commercial,64784,PANELTO FOODS LTD,FRES H.,Fresh,BAKERY&DAIR Y.,Bakery & Dairy,IN STORE BA K.,Instore Bakery,...,16,0,NaN,NaN,NaN,no match,12575979.0,PANELTO FOODS LTDFBAKERYORDERABL.,PANELTO FOODS LTDFBAKERYORDERABL.,True
19,11575CHILLED SNACK S.,Commercial,11575,PORK FARMS LIMITED(VF,FRES H.,Fresh,PREPARED FOOD S.,Prepared Foods,CHILLE D.,Chilled,...,3,0,NaN,NaN,NaN,no match,18022.0,ADDO FOOD GROUPCHILLED SNACK S.,Kerry Foods Ltd,False
35,61961PREPVE G.,Commercial,61961,KANES FOODS LTD (FF,FRES H.,Fresh,PRODUC E.,Produce,PREPARE D.,"Prepared Salad, Fruit & Veg",...,3,0,NaN,NaN,NaN,no match,19.0,KANES FOODS LTDPREPVE G.,KANES FOODS LTDPREPVE G.,True
77,67723DRIEDFRUITNUT S.,Commercial,67723,MAITRE PRUNILLE S.A.S.(MP(EUR,FRES H.,Fresh,PRODUC E.,Produce,PREPARE D.,"Prepared Salad, Fruit & Veg",...,5,1,NaN,NaN,NaN,no match,1555657.0,MAITRE PRUNILLEDRIEDFRUITNUT S.,MAITRE PRUNILLEDRIEDFRUITNUT S.,True


In [51]:
non_matched[non_matched['flag']=='False']
non_matched.loc[(non_matched.flag == 'False'),'Supplier Name from TQC']=non_matched['TQC_Supplier_Name_2']
non_matched[non_matched['flag']=='False']
non_matched.to_excel('df_10.xlsx')

In [70]:
total_matched_combo_2=df_Original_List[['Supplier_Name','Junior_Buyer','Supplier Name from TQC','Parent_Supplier_Name']][(df_Original_List['OBL Count']>0) & (df_Original_List['Supplier Name from TQC'].notnull())].drop_duplicates()
len(total_matched_combo)

1424

In [71]:
total_matched_combo_2.head(2)
tqc['Supplier_Name'].head(2)

0    Bakkavor
1    Bakkavor
Name: Supplier_Name, dtype: object

In [72]:
total_matched_combo_2=total_matched_combo_2.merge(tqc[['Comm_Supplier_Name','Junior_Buyer','Site_Name']],left_on=['Supplier_Name','Junior_Buyer'],right_on=['Comm_Supplier_Name','Junior_Buyer'],how='left')
total_matched_combo_2.head(2)

,Supplier_Name,Junior_Buyer,Supplier Name from TQC,Parent_Supplier_Name,Comm_Supplier_Name,Site_Name
0,FRIESLANDCAMPINANEDERLAND BV,SPECIALCHEES E.,FrieslandCampina,FRIESLAND FOODS,FRIESLANDCAMPINANEDERLAND BV,FrieslandCampina Leerdam
1,FRIESLANDCAMPINANEDERLAND BV,SPECIALCHEES E.,FrieslandCampina,FRIESLAND FOODS,FRIESLANDCAMPINANEDERLAND BV,FrieslandCampina Leerdam


In [73]:
match=[]
match_2=[]
match_3=[]
for k in range(0,len(total_matched_combo_2)):
    match.append(fuzz.token_sort_ratio(total_matched_combo_2['Supplier_Name'].iloc[k],total_matched_combo_2['Supplier Name from TQC'].iloc[k]))
    match_2.append(fuzz.token_sort_ratio(total_matched_combo_2['Supplier_Name'].iloc[k],total_matched_combo_2['Site_Name'].iloc[k]))
    match_3.append(fuzz.token_sort_ratio(total_matched_combo_2['Supplier Name from TQC'].iloc[k],total_matched_combo_2['Parent_Supplier_Name'].iloc[k]))
    
total_matched_combo_2['Match1']=match
total_matched_combo_2['Match2']=match_2
total_matched_combo_2['Match3']=match_3

total_matched_combo_2.head()


,Supplier_Name,Junior_Buyer,Supplier Name from TQC,Parent_Supplier_Name,Comm_Supplier_Name,Site_Name,Match1,Match2,Match3
0,FRIESLANDCAMPINANEDERLAND BV,SPECIALCHEES E.,FrieslandCampina,FRIESLAND FOODS,FRIESLANDCAMPINANEDERLAND BV,FrieslandCampina Leerdam,73,77,58
1,FRIESLANDCAMPINANEDERLAND BV,SPECIALCHEES E.,FrieslandCampina,FRIESLAND FOODS,FRIESLANDCAMPINANEDERLAND BV,FrieslandCampina Leerdam,73,77,58
2,FRIESLANDCAMPINANEDERLAND BV,SPECIALCHEES E.,FrieslandCampina,FRIESLAND FOODS,FRIESLANDCAMPINANEDERLAND BV,FrieslandCampina Leerdam,73,77,58
3,BAKKAVOR SPALDING(FF(VF,CHILLED PIZZ A.,Bakkavor,BAKKAVOR,BAKKAVOR SPALDING(FF(VF,Bakkavör Salads Spalding - Delicatessen,52,58,100
4,BAKKAVOR SPALDING(FF(VF,CHILLED PIZZ A.,Bakkavor,BAKKAVOR,BAKKAVOR SPALDING(FF(VF,Bakkavör Salads Spalding - Delicatessen,52,58,100


In [74]:
total_matched_combo_2.to_excel('df_6.xlsx')